In [1]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from string import punctuation
from autocorrect import spell

snowball_stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()

class Preprocess:
    def __int__(self):
        pass

    def autospell(self,text):
       
        #correct the spelling of the word.
       
        spells = [spell(w) for w in (nltk.word_tokenize(text))]
        return " ".join(spells)

    def to_lower(self,text):
        
       # :param text:
        #:return:
         #   Converted text to lower case as in, converting "Hello" to "hello" or "HELLO" to "hello".
        
        return text.lower()

    def remove_numbers(self,text):
        
        #take string input and return a clean text without numbers.
       # Use regex to discard the numbers.
        
        output = ''.join(c for c in text if not c.isdigit())
        return output

    def remove_punct(self,text):
      
        #take string input and clean string without punctuations.
        #use regex to remove the punctuations.
       
        return ''.join(c for c in text if c not in punctuation)

    def remove_Tags(self,text):
        
        #take string input and clean string without tags.
        #use regex to remove the html tags.
        
        cleaned_text = re.sub('<[^<]+?>', '', text)
        return cleaned_text

    def sentence_tokenize(self,text):
       
        #take string input and return list of sentences.
       # use nltk.sent_tokenize() to split the sentences.
        
        sent_list = []
        for w in nltk.sent_tokenize(text):
            sent_list.append(w)
        return sent_list

    def word_tokenize(self,text):
        
        #:param text:
        #:return: list of words
       
        return [w for sent in nltk.sent_tokenize(text) for w in nltk.word_tokenize(sent)]

    def remove_stopwords(self,sentence):
    
        #removes all the stop words like "is,the,a, etc."
      
        stop_words = stopwords.words('english')
        return ' '.join([w for w in nltk.word_tokenize(sentence) if not w in stop_words])

    def stem(self,text):
       
        #:param word_tokens:
        #:return: list of words
        
        stemmed_word = [snowball_stemmer.stem(word) for sent in nltk.sent_tokenize(text)for word in nltk.word_tokenize(sent)]
        return " ".join(stemmed_word)

    def lemmatize(self,text):
        lemmatized_word = [wordnet_lemmatizer.lemmatize(word)for sent in nltk.sent_tokenize(text)for word in nltk.word_tokenize(sent)]
        return " ".join(lemmatized_word)


    def preprocess(self,text):
        lower_text = self.to_lower(text)
        sentence_tokens = self.sentence_tokenize(lower_text)
        word_list = []
        for each_sent in sentence_tokens:
            lemmatizzed_sent = self.lemmatize(each_sent)
            clean_text = self.remove_numbers(lemmatizzed_sent)
            clean_text = self.remove_punct(clean_text)
            clean_text = self.remove_Tags(clean_text)
            clean_text = self.remove_stopwords(clean_text)
            word_tokens = self.word_tokenize(clean_text)
            for i in word_tokens:
                word_list.append(i)
        return word_list

In [2]:
file = open("/home/mohit/snomed.txt",'r')
final_sensitive_list = []

for line in file:
    strip_line = line.strip()
    final_sensitive_list.append(strip_line)
file.close()
final_sensitive_list

['hiv/aids',
 'epidemiology of hiv/aids',
 'timeline of hiv/aids',
 'hiv/aids in the united states',
 'aids (disambiguation)',
 'list of countries by hiv/aids adult prevalence rate',
 'hiv/aids in africa',
 'hiv/aids denialism',
 'aids (computer virus)',
 'hiv/aids research',
 'hiv',
 'hiv/aids',
 'epidemiology of hiv/aids',
 'subtypes of hiv',
 'history of hiv/aids',
 'hiv (disambiguation)',
 'diagnosis of hiv/aids',
 'list of countries by hiv/aids adult prevalence rate',
 'hiv/aids in africa',
 'management of hiv/aids',
 'tb',
 'tuberculosis',
 'tampa bay rays',
 'terabyte',
 'socata tb family',
 'multi-drug-resistant tuberculosis',
 'tuberculosis radiology',
 'tuberculosis in india',
 'directly observed treatment, short-course',
 'tb alert',
 'cancer',
 'breast cancer',
 'colorectal cancer',
 'lung cancer',
 'pancreatic cancer',
 'not cancer',
 'cancer (astrology)',
 'prostate cancer',
 'esophageal cancer',
 'stomach cancer',
 'bipolar disorder',
 'bipolar',
 'bipolar ii disorder',


In [3]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd
import pickle
import spacy


from scispacy.abbreviation import AbbreviationDetector
import en_ner_bionlp13cg_md
from scispacy.abbreviation import AbbreviationDetector
import en_ner_bc5cdr_md
import en_ner_bionlp13cg_md
import re
import datefinder as df
import datetime as dt

In [4]:
def bin_avg(s):
    ag=s
    i=1
    j=10
    while i<j and j<150:
        if ag in range(i,j+1):
            return str((i+j)/2)
           
        i+=10
        j+=10
def sanitization(file):
    #extract date
    s1=re.findall(r'(?:\d{1,2}[a-z]* )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}[a-z]*, )?\d{4}',file)
    s=re.findall(r'\d{1,2}[/]\d{1,2}[/]\d{2,4}',file)
    #month=re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',file)
    #print(s1)
    #print(s)
    
    
    #replace date
    for i in range(len(s)):
        y="".join(s[i])
        y_1=y
        y=y.replace('0/','1/')
        mat=list(df.find_dates(y))
        rep=str(dt.datetime.date(mat[0]).year)
        rep='['+rep+']'
        file=file.replace(y_1,rep)
    if len(s1)>0:    
        for i in range(len(s1)):
            y1="".join(s1[i])
            mat1=list(df.find_dates(s1[i]))
            #print(mat)
            rep1=str(dt.datetime.date(mat1[0]).year)
            rep1='['+rep1+']'
            file=file.replace(y1,rep1)
    
    #extract and replace Record Number
    r=file.split()
    file=file.replace(r[1],'#####')
    
    #extract and replace patient and doctor name
    name=re.findall(r'Mr\. \w[A-Za-z]+|Mrs\. \w[A-Za-z]+',file)
    #print(name)
    for i in range(len(name)):
        file=file.replace(name[i],"[PERSON]")
    
    doctor=re.findall(r'Dr\. \w[A-Za-z]+',file)
    #print(doctor)
    for i in range(len(doctor)):
        file=file.replace(doctor[i],"[DOCTOR]")
    
    #extract and remove unwanted data
    i=2
    while(r[i]!='Admission'):
        #print(r[i])
        file=file.replace(r[i],'*',1)
        i+=1
   
    #extract and replace dictated fields and attending fields
    d=re.findall(r'Dictated By:\s[A-Za-z]\s.|DICTATED By:\s[A-Za-z]\s.',file)
    #print(d)
    for i in range(len(d)):
        file=file.replace(d[i],'Dictated By: [DOCTORS]')
    
    a=re.findall(r'Attending:\s[A-Za-z]\s.|ATTENDING:\s[A-Za-z]\s.|ENTERED BY:\s[A-Za-z]\s.',file)
    #print(a)
    for i in range(len(a)):
        file=file.replace(a[i],'Attending: [PERSONS]')


    id=re.findall(r'Dictation ID\s.*',file)
    for i in range(len(id)):
        file=file.replace(id[i],'Dictation ID ****')
    
    #extract age
    age=re.findall(r'[0-9]+[-\s]year[-\s]|[0-9]+[-\s]years[-\s]',file)
    #print(age)
    for i in range(len(age)):
        a=str(age[i])
        a1=list(age[i])
        age_years="".join(a1[0:2])
        dob=bin_avg(float(age_years))
        years='['+dob+']'
        file=file.replace(age[i],years+' years ')
        
    #extract and replace Phone number
    phno=re.findall(r'\d{3}[-\s]+\d{3}[-\s]+\d{4}',file)
    #print(phno)
    for i in range(len(phno)):
        file=file.replace(phno[i],"[PHONE NUMBER]")
        
    #extract and replace Batch and Index Number
    batch=re.findall(r'Batch:\s[0-9]+\sIndex No\.[-\s][A-Za-z0-9]+',file)
    #print(batch)
    for i in range(len(batch)):
        file=file.replace(batch[i],'Batch: ** \tIndex No. ***')
    
    #extract and replace eScription document
    doc=re.findall(r'eScription document:\s.*',file)
    for i in range(len(doc)):
        file=file.replace(doc[i],"eScription document: **")
   
    return(file)

#read and write the file 
f=open(r"finaldocuments/6637",'r')
file=f.read()

#call the sanitization function
synthetic_data=sanitization(file)#the modified data is stored in synthetic data....
print(synthetic_data)    

RECORD #####
* * * * * * * * * * * * * * * * * * * * * * Admission Date: [2006] Report Status: Signed
Discharge Date: [2006]
ATTENDING: LANGILL , ADOLPH M.D.
CODE STATUS: Full code.
The patient was admitted to the Renal Transplant PA Service.
ADMITTING DIAGNOSES:
1. Acute renal failure.
2. Congestive heart failure.
3. End-stage renal disease , status post renal transplant.
PRINCIPAL DISCHARGE DIAGNOSES:
1. Acute renal failure requiring dialysis initiation.
2. CHF -improved.
3. End-stage renal disease , status post failed renal transplant.
OTHER DIAGNOSES AND CONDITIONS AFFECTING TREATMENT OR STAY:
Hypertension , congestive heart failure , chronic renal
dysfunction , renal transplant , CVA and pulmonary hypertension.
OPERATIONS AND PROCEDURE: Not applicable.
OTHER TREATMENTS AND PROCEDURES: Initiation of dialysis , IV
diuretics.
BRIEF SUMMARY OF HOSPITAL COURSE:
CHIEF COMPLAINT: Shortness of breath.
HISTORY OF PRESENT ILLNESS: [PERSON] is a [55.5] years old male
with history of end-stag

In [5]:


#function to find the range of age
def range_age(s):
        ag=s
        i=1
        j=10
        x=''
        while i<j and j<150:
            if ag in range(i,j+1):
                x='['+str(i)+'-'+str(j)+']'
                return x
           
            i+=10
            j+=10
    

#main function to extract and replace
def Generalization(file):
    s1=re.findall(r'(?:\d{1,2}[a-z]* )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}[a-z]*, )?\d{4}',file)
    s=re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',file)
     #replace date
    for i in range(len(s)):
        file=file.replace(s[i],'[DATE]')
    if len(s1)>0:
       for i in range(len(s1)):
         file=file.replace(s1[i],'[DATE]') 
    
    #extract and replace Record Number
    r=file.split()
    file=file.replace(r[1],'#####')
    
    #extract and replace patient and doctor name
    name=re.findall(r'Mr\. \w[A-Za-z]+|Mrs\. \w[A-Za-z]+',file)
    for i in range(len(name)):
        file=file.replace(name[i],'[PERSON]')
    
    name=re.findall(r'DR\. \w[A-Za-z]+|Mrs\. \w[A-Za-z]+',file)
    for i in range(len(name)):
        file=file.replace(name[i],'[DOCTOR]')
        
    #extract and replace Phone number
    phno=re.findall(r'\d{3}[-\s]+\d{3}[-\s]+\d{4}',file)
    for i in range(len(phno)):
        file=file.replace(phno[i],'[PHONE NUMBER]')
    
    #extract age
    age=re.findall(r'[0-9]+[-\s]year[-\s]|[0-9]+[-\s]years[-\s]',file)
    for i in range(len(age)):
        a=str(age[i])
        a1=list(a)
        age_years="".join(a1[0:2])
        dob=range_age(float(age_years))
        file=file.replace(age[i],dob+'years')
    
    #extract and remove unwanted data
    i=2
    while(r[i]!='Admission'):
        file=file.replace(r[i],'*',1)
        i+=1
        
    #extract and replace dictated fields and attending fields
    d=re.findall(r'Dictated By:\s[A-Za-z]*\s.*| Dictated By:\s[A-Za-z]*\s.',file)
    for i in range(len(d)):
        file=file.replace(d[0],'Dictated By: [DOCTORS]')
        
    a=re.findall(r'Attending:\s[A-Za-z]*\s.*| ATTENDING:\s[A-Za-z]*\s.*',file)
    for i in range(len(a)):
        file=file.replace(a[0],'Attending: [PERSONS] ')
        
    a1=re.findall(r'ATTENDING:\s[A-Za-z]*\s.*',file)
    for i in range(len(a1)):
        file=file.replace(a1[0],'ATTENDING: [PERSONS] ')
    
    e=re.findall(r'ENTERED BY:\s[A-Za-z]*\s.*',file)
    for i in range(len(e)):
        file=file.replace(e[0],'ENTERED BY: [PERSONS] ')
    
    id=re.findall(r'Dictation ID\s.*',file)
    for i in range (len(id)):
        file=file.replace(id[i],'Dictation ID: ********')
        
    #extract and replace Batch and Index Number
    batch=re.findall(r'Batch:\s[0-9]+\sIndex No\.[-\s][A-Za-z0-9]+',file)
    for i in range(len(batch)):
        file=file.replace(batch[i],'batch:''****'+'\tIndex No.'' *********')
  
    return(file)

f=open(r"finaldocuments/6637",'r')
file=f.read()

General_data=Generalization(file)#the modified data is stored in General data....
print(General_data) 

RECORD #####
* * * * * * * * * * * * * * * * * * * * * * Admission Date: [DATE] Report Status: Signed
Discharge Date: [DATE]
ATTENDING: [PERSONS] 
CODE STATUS: Full code.
The patient was admitted to the Renal Transplant PA Service.
ADMITTING DIAGNOSES:
1. Acute renal failure.
2. Congestive heart failure.
3. End-stage renal disease , status post renal transplant.
PRINCIPAL DISCHARGE DIAGNOSES:
1. Acute renal failure requiring dialysis initiation.
2. CHF -improved.
3. End-stage renal disease , status post failed renal transplant.
OTHER DIAGNOSES AND CONDITIONS AFFECTING TREATMENT OR STAY:
Hypertension , congestive heart failure , chronic renal
dysfunction , renal transplant , CVA and pulmonary hypertension.
OPERATIONS AND PROCEDURE: Not applicable.
OTHER TREATMENTS AND PROCEDURES: Initiation of dialysis , IV
diuretics.
BRIEF SUMMARY OF HOSPITAL COURSE:
CHIEF COMPLAINT: Shortness of breath.
HISTORY OF PRESENT ILLNESS: [PERSON] is a [51-60]yearsold male
with history of end-stage renal dise

In [6]:
def summarization(filename):
    text=''
    with open(filename,"r") as a_file:
        for line in a_file:
            stripped_line = line.replace("\n"," ")
            text+=stripped_line
    def content(pattern):
    #pattern='PRINCIPAL DIAGNOSIS:(.*?)(:)'
        principal_diagnosis = re.search(pattern,text).group(0)
#print(res)
#Identifying the last label
        result = re.search('[ a-zA-Z]+\:$',principal_diagnosis).group(0)
#res1
#splitting each sentence based on full stop
        result_1=principal_diagnosis.split(".")
#result_1
#creating a list having only principal diagnosis
        text_list=[]
        for i in result_1:
            if i != result:
                text_list.append(i)
#text_list
        final_result = '.'.join(text_list)
        final= final_result+'.'
        return final
    
    
    from IPython.display import Markdown, display
    def printmd(string):
        display(Markdown(string))
        
    import re
    d1 = re.findall(r"Admission Date: [\d]{1,2}/[\d]{1,2}/[\d]{4}", text)
    for s in d1:
        printmd(s)
    d2 = re.findall(r"Discharge Date: [\d]{1,2}/[\d]{1,2}/[\d]{4}", text)
    for s in d2:
        printmd(s) 
    p_d=""
    h_p_i=""
    p_m_h=""
    m_o_a=""
    allergies=""
    p_e_a=""
    if 'PRINCIPAL DIAGNOSIS ON DISCHARGE:' in text:
        p_d = 'PRINCIPAL DIAGNOSIS ON DISCHARGE:(.*?)(:)'
        p_d = content(p_d)
    if 'HISTORY OF PRESENT ILLNES:' in text:
        h_p_i='HISTORY OF PRESENT ILLNESS:(.*?)(:)'
        h_p_i=content(h_p_i)
    if 'PAST MEDICAL HISTORY:' in text:
        p_m_h='PAST MEDICAL HISTORY:(.*?)(:)'
        p_m_h=content(p_m_h)
    if 'MEDICATIONS ON ADMISSION:' in text:
        m_o_a='MEDICATIONS ON ADMISSION:(.*?)(:)'
        m_o_a=content(m_o_a)
    if 'ALLERGIES' in text:
        allergies='ALLERGIES:(.*?)(:)'
        allergies=content(allergies)
    if 'PHYSICAL EXAMINATION ON ADMISSION:' in text:
        p_e_a='PHYSICAL EXAMINATION ON ADMISSION:(.*?)(:)'
        p_e_a=content(p_e_a)
    res = str(d1)+"\n"+str(d2)+"\n"+p_d+"\n"+h_p_i+"\n"+p_m_h+"\n"+m_o_a+"\n"+allergies+"\n"+p_e_a
    res1 = res.split('\n')
    with open('summerization.txt',"w") as f:
        for lis in res1:
            if lis != "":
                f.write('%s\n'%lis)
    return res1

In [7]:
from flask import *  
import spacy
import os
from werkzeug.utils import secure_filename
app = Flask(__name__, template_folder='templates')
UPLOAD_FOLDER = 'home/mohit/Documents'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER# Upload API
global res
def check_sensitive(file):
        f=open(file)
        y=f.read()

        pr = Preprocess()
        res=pr.preprocess(y)
        y=" ".join(res)
        y
        nlp_bc = en_ner_bc5cdr_md.load()
        nlp_bi = en_ner_bionlp13cg_md.load()
        doc_bio = nlp_bi(y)
        doc_bc = nlp_bc(y)
    
        doc_bc_list = [item for i in doc_bc.ents for item in i]
    
    #spacy.displacy.render(doc, style='ent', jupyter=True)
        doc_list_bc = [item for i in doc_bio.ents for item in i]
    
        sen_list =  doc_bc_list+doc_list_bc#doc_med7_list 
        sensitive = False
        sen=[]
        for item in sen_list:
            term1 =  str(item)
            if  term1 in final_sensitive_list:
                sensitive=True
                sen.append(term1)
                
            
        if sensitive is True:
            f=open(file,'r')
            file=f.read()
            san_res = sanitization(file)#sanitize the file
            split_line = san_res.split('\n')
            with open('senitization.txt',"w") as f:
                for lis in split_line:
                    f.write('%s\n'%lis)
    
            sum_res = summarization('senitization.txt')
            
            return sum_res
        else:
            sum_res = summarization(file)
            return sum_res
    
 


@app.route('/uploadfile', methods=['GET', 'POST'])  
def upload():  
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            print('no file')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            print('no filename')
            return redirect(request.url)
        else:
            file = secure_filename(file.filename)
            
            file = 'finaldocuments/'+file
            print("saved file successfully")
      #send file name as parameter to downlad
            
            res = check_sensitive(file)
            with open('finaldownloaded/summerization.txt',"w") as f:
                for lis in res:
                    f.write('%s\n'%lis)
            filename = 'summerization.txt'
            return redirect('/nonsen_download')
    return render_template("upload_file.html")  
 
    
@app.route('/nonsen_downloads')#non sensitive file download
def downloadFile ():
    #For windows you need to use drive name [ex: F:/Example.pdf]
    
    path = "finalize.txt"
    return send_file(path, as_attachment=True)

@app.route('/sen_downloads')#sensitive file download
def downloadFile1 ():
    #For windows you need to use drive name [ex: F:/Example.pdf]
    path = "finalize.txt"
    return send_file(path, as_attachment=True)

@app.route("/downloadfile/<filename>", methods = ['GET'])
def download_file(filename):
    f = open('finaldocuments/'+filename,'w')
    
    with open('finalize.txt',"w") as f:
                for lis in res:
                    f.write('%s\n'%lis)
    filename = 'finalize.txt'
    return render_template('download.html',value=filename) 

@app.route('/return-files/<filename>')
def return_files_tut(filename):
    file_path = UPLOAD_FOLDER +"/" + filename
    
            
    return send_file(file_path, as_attachment=True, attachment_filename='')
@app.route('/')  
def upload1():  
    return render_template("file_upload_form.html")  

@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        f = request.files['file']  
        #file = f.filename+str(f) 
        print('File uploaded succesfully')
        s = str(f.filename)
        file = "finaldocuments/"+s
        nam = check_sensitive(file)
       

        return render_template('success.html',value=nam) 
  


if __name__ == '__main__':  
    app.run(debug = True, port="5008" , use_reloader=False)  


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5008/ (Press CTRL+C to quit)


In [59]:
from flask import *  
import spacy


def check_sensitive(file):
        f=open(file)
        y=f.read()

        pr = Preprocess()
        res=pr.preprocess(y)
        y=" ".join(res)
        y
        nlp_bc = en_ner_bc5cdr_md.load()
        nlp_bi = en_ner_bionlp13cg_md.load()
        doc_bio = nlp_bi(y)
        doc_bc = nlp_bc(y)
    
        doc_bc_list = [item for i in doc_bc.ents for item in i]
    
    #spacy.displacy.render(doc, style='ent', jupyter=True)
        doc_list_bc = [item for i in doc_bio.ents for item in i]
    
        sen_list =  doc_bc_list+doc_list_bc#doc_med7_list 
        sensitive = False
        sen=[]
        for item in sen_list:
            term1 =  str(item)
            if  term1 in final_sensitive_list:
                sensitive=True
                sen.append(term1)
                
            
        if sensitive is True:
            f=open(file,'r')
            file=f.read()
            san_res = sanitization(file)#sanitize the file
            split_line = san_res.split('\n')
            with open('senitization.txt',"w") as f:
                for lis in split_line:
                    f.write('%s\n'%lis)
    
            sum_res = summarization('senitization.txt')
            
            return sum_res
        else:
            sum_res = summarization(file)
            return sum_res
    
app = Flask(__name__)  
 
@app.route('/')  
def upload():  
    return render_template("file_upload_form.html")  
 
    


@app.route('/sen_download')#sensitive file download
def downloadFile1 ():
    #For windows you need to use drive name [ex: F:/Example.pdf]
    path = "summerization.txt"
    return send_file(path, as_attachment=True)

@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        f = request.files['file']  
        #file = f.filename+str(f) 
        
        s = str(f.filename)
        file = "finaldocuments/"+s
        nam = check_sensitive(file)
       
        return render_template("success.html", name = nam)  

@app.route('/return-files/summerization.txt')
def return_files_tut(filename):
    file_path = "summerization.txt"
    
            
    return send_file(file_path, as_attachment=True, attachment_filename='')
  

if __name__ == '__main__':  
    app.run(debug = True, port="5008" , use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2020 22:02:53] "GET / HTTP/1.1" 200 -


Admission Date: 4/7/1996

Discharge Date: 6/21/1996

127.0.0.1 - - [27/Oct/2020 22:04:00] "POST /success HTTP/1.1" 200 -


In [7]:
from flask import *  
import spacy


def check_sensitive(file):
        f=open(file)
        y=f.read()

        pr = Preprocess()
        res=pr.preprocess(y)
        y=" ".join(res)
        y
        nlp_bc = en_ner_bc5cdr_md.load()
        nlp_bi = en_ner_bionlp13cg_md.load()
        doc_bio = nlp_bi(y)
        doc_bc = nlp_bc(y)
    
        doc_bc_list = [item for i in doc_bc.ents for item in i]
    
    #spacy.displacy.render(doc, style='ent', jupyter=True)
        doc_list_bc = [item for i in doc_bio.ents for item in i]
    
        sen_list =  doc_bc_list+doc_list_bc#doc_med7_list 
        sensitive = False
        sen=[]
        for item in sen_list:
            term1 =  str(item)
            if  term1 in final_sensitive_list:
                sensitive=True
                sen.append(term1)
                
            
        if sensitive is True:
            
            return True
        else:
            return False
    
app = Flask(__name__)  
 
@app.route('/')  
def upload():  
    return render_template("file_upload_form.html")  
 
    


@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        f = request.files['file']  
        #file = f.filename+str(f) 
        print('file uploaded successfully')
        s = str(f.filename)
        file = "finaldocuments/"+s
        res = check_sensitive(file)
        if res == True:
            return render_template("ananimization.html", name = file)
        else:
            res = summarization(file)
            return render_template("download.html", name = file)  
  
@app.route('/return-files')
def return_files_tut():
    file_path = "summerization.txt"
    
            
    return send_file(file_path, as_attachment=True)
if __name__ == '__main__':  
    app.run(debug = True, port="5008" , use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2020 11:52:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 11:52:32] "GET /favicon.ico HTTP/1.1" 404 -


file uploaded successfully


Admission Date: 9/12/2005

Discharge Date: 10/16/2005

127.0.0.1 - - [28/Oct/2020 11:53:24] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 11:53:26] "GET /return-files HTTP/1.1" 200 -


In [23]:
with open('summerization.txt',"w") as f:
    for lis in res1:
        
        f.write('%s\n'%lis)

In [34]:
with open('summerization.txt',"r") as f:
    for lis in f:
        print(lis)

HISTORY OF PRESENT ILLNESS: Mr. Midget is a 56-year-old male with a past medical history significant for inferior myocardial infarction in 1995 , which was treated with angioplasty at that time. Since then , he has been stable on medical management until approximately two months prior to this admission , when he began noting anterior chest pain with exertion. He underwent a stress test , which was found to be abnormal , and was referred for cardiac catheterization. Cardiac catheterization was performed on 10/3/06 at Au Hospital . This study revealed a 70% proximal stenosis of his left anterior descending coronary artery , a 90% mid segment stenosis of his first diagonal coronary artery , a 50% mid segment stenosis of his circumflex coronary artery , a 90% ostial stenosis of his first obtuse marginal , a 100% proximal stenosis of his right coronary artery as well as a 30% ostial stenosis of the left main coronary artery. Collateral flow was noted from the left anterior descending to the

In [10]:
!conda install datefinder

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - datefinder

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [13]:
import re
import datefinder as df
import datetime as dt

#function to replace age as bin_avg of age
def bin_avg(s):
    ag=s
    i=1
    j=10
    while i<j and j<150:
        if ag in range(i,j+1):
            return str((i+j)/2)
           
        i+=10
        j+=10

#main function to extract and replace
def sanitization(file):
    #extract date
    s1=re.findall(r'(?:\d{1,2}[a-z]* )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}[a-z]*, )?\d{4}',file)
    s=re.findall(r'\d{1,2}[/]\d{1,2}[/]\d{2,4}',file)
    #month=re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*',file)
    #print(s1)
    #print(s)

    #replace date
    for i in range(len(s)):
        y="".join(s[i])
        y_1=y
        y=y.replace('0/','1/')
        mat=list(df.find_dates(y))
        rep=str(dt.datetime.date(mat[0]).year)
        rep='['+rep+']'
        file=file.replace(y_1,"\033[1;31;47m {}\033[00m".format(rep))
    if len(s1)>0:    
        for i in range(len(s1)):
            y1="".join(s1[i])
            mat1=list(df.find_dates(s1[i]))
            #print(mat)
            rep1=str(dt.datetime.date(mat1[0]).year)
            rep1='['+rep1+']'
            file=file.replace(y1,"\033[1;31;47m {}\033[00m".format(rep1))
    
    #extract and replace Record Number
    r=file.split()
    file=file.replace(r[1],'\033[1;31;91m {}\033[00m'.format('#####'))
    
    #extract and replace patient and doctor name
    name=re.findall(r'Mr\. \w[A-Za-z]+|Mrs\. \w[A-Za-z]+',file)
    #print(name)
    for i in range(len(name)):
        file=file.replace(name[i],"\033[1;31;47m {}\033[00m".format("[PERSON]"))
    doctor=re.findall(r'Dr\. \w[A-Za-z]+',file)
    #print(doctor)
    for i in range(len(doctor)):
        file=file.replace(doctor[i],"\033[1;31;47m {}\033[00m".format("[DOCTOR]"))
    
    #extract and remove unwanted data
    i=2
    while(r[i]!='Admission'):
        #print(r[i])
        file=file.replace(r[i],'\033[1;31;91m {}\033[00m'.format('*'),1)
        i+=1
   
    #extract and replace dictated fields and attending fields
    d=re.findall(r'Dictated By:\s[A-Za-z]*\s.*|DICTATED By:\s[A-Za-z]*\s.*',file)
    #print(d)
    for i in range(len(d)):
        file=file.replace(d[i],'Dictated By: \033[1;31;47m {}\033[00m'.format('[DOCTORS]'))
    a=re.findall(r'Attending:\s[A-Za-z]*\s.*|ATTENDING:\s[A-Za-z]*\s.*|ENTERED BY:\s[A-Za-z]*\s.*',file)
    #print(a)
    for i in range(len(a)):
        file=file.replace(a[i],'Attending: \033[1;31;47m {}\033[00m'.format("[PERSONS]"))


    id=re.findall(r'Dictation ID\s.*',file)
    for i in range(len(id)):
        file=file.replace(id[i],'Dictation ID \033[1;31;91m {}\033[00m'.format('********'))
    
    #extract age
    age=re.findall(r'[0-9]+[-\s]year[-\s]|[0-9]+[-\s]years[-\s]',file)
    #print(age)
    for i in range(len(age)):
        a=str(age[i])
        a1=list(age[i])
        age_years="".join(a1[0:2])
        dob=bin_avg(float(age_years))
        years='['+dob+']'
        file=file.replace(age[i],'\033[1;31;47m {}\033[00m'.format(years)+' years ')
        
    #extract and replace Phone number
    phno=re.findall(r'\d{3}[-\s]+\d{3}[-\s]+\d{4}',file)
    #print(phno)
    for i in range(len(phno)):
        file=file.replace(phno[i],"\033[1;31;47m {}\033[00m".format("[PHONE NUMBER]"))
        
    #extract and replace Batch and Index Number
    batch=re.findall(r'Batch:\s[0-9]+\sIndex No\.[-\s][A-Za-z0-9]+',file)
    #print(batch)
    for i in range(len(batch)):
        file=file.replace(batch[i],'Batch:\033[1;31;91m {}\033[00m'.format("****")+ '\tIndex No. \033[1;31;91m {}\033[00m'.format("*****"))
    
    #extract and replace eScription document
    doc=re.findall(r'eScription document:\s.*',file)
    for i in range(len(doc)):
        file=file.replace(doc[i],"eScription document: \033[1;31;47m {}\033[00m".format("******"))
   
    return(file)

#read and write the file 
f=open(r"finaldocuments/6637",'r')
file=f.read()

#call the sanitization function
synthetic_data=sanitization(file)#the modified data is stored in synthetic data....
print(synthetic_data)    

RECORD  #####
 *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  * Admission Date:  [2006] Report Status: Signed
Discharge Date:  [2006]
Attending:  [PERSONS]
CODE STATUS: Full code.
The patient was admitted to the Renal Transplant PA Service.
ADMITTING DIAGNOSES:
1. Acute renal failure.
2. Congestive heart failure.
3. End-stage renal disease , status post renal transplant.
PRINCIPAL DISCHARGE DIAGNOSES:
1. Acute renal failure requiring dialysis initiation.
2. CHF -improved.
3. End-stage renal disease , status post failed renal transplant.
OTHER DIAGNOSES AND CONDITIONS AFFECTING TREATMENT OR STAY:
Hypertension , congestive heart failure , chronic renal
dysfunction , renal transplant , CVA and pulmonary hypertension.
OPERATIONS AND PROCEDURE: Not applicable.
OTHER TREATMENTS AND PROCEDURES: Initiation of dialysis , IV
diuretics.
BRIEF SUMMARY OF HOSPITAL COURSE:
CHIEF COMPLAINT: Shortness of breath.
HISTORY OF PRESENT ILLNESS:  [PERSON] is a  [55.5] years old male
with 

In [12]:
!pip install datefinder

  Using cached datefinder-0.7.1-py2.py3-none-any.whl (10 kB)


In [40]:
f=open("finaldocuments/185795",'r')
file=f.read()

san_res = sanitization(file)
f = open('sanitization.txt','w')
f.write(san_res)
split_line = san_res.split('\n')
f = open('sanitization.txt','r')
f.read()
    
sum_res = summarization('sanitization.txt')
sum_res

"RECORD #####\n* * * * * * * * * * * * * * * * * * * * Admission Date: [1999] Report Status: Signed\nDischarge Date: [1999]\nPRINCIPAL DIAGNOSIS: 1. Stroke.\nSECONDARY DIAGNOSIS: 2. Coronary artery disease.\n3. Hypertension. 4. Hypercholesterolemia.\n5. Type II diabetes mellitus. 6. Gastroesophageal reflux\ndisease.\nPRINCIPAL PROCEDURES AND OPERATIONS: The patient had a head MRI ,\nan echocardiogram , and head\nCT scan.\nHISTORY OF PRESENT ILLNESS: This [65.5] years old male was in his usual\nstate of health until the morning of\nadmission when he woke up with speech difficulty noticed by his\nwife. The patient was noted to make paraphasic errors. In\naddition , there was a question of a right facial droop. The\npatient presented to the emergency room. There were no other\nneurologic symptoms.\nPHYSICAL EXAMINATION: Physical examination on admission revealed\npulse 78 , temperature 98.8 , and blood\npressure 160/78. The patient was in no apparent distress. The\nhead and neck examinati

In [31]:
san_res

"RECORD #####\n* * * * * * * * * * * * * * * * * * * * Admission Date: [1999] Report Status: Signed\nDischarge Date: [1999]\nPRINCIPAL DIAGNOSIS: 1. Stroke.\nSECONDARY DIAGNOSIS: 2. Coronary artery disease.\n3. Hypertension. 4. Hypercholesterolemia.\n5. Type II diabetes mellitus. 6. Gastroesophageal reflux\ndisease.\nPRINCIPAL PROCEDURES AND OPERATIONS: The patient had a head MRI ,\nan echocardiogram , and head\nCT scan.\nHISTORY OF PRESENT ILLNESS: This [65.5] years old male was in his usual\nstate of health until the morning of\nadmission when he woke up with speech difficulty noticed by his\nwife. The patient was noted to make paraphasic errors. In\naddition , there was a question of a right facial droop. The\npatient presented to the emergency room. There were no other\nneurologic symptoms.\nPHYSICAL EXAMINATION: Physical examination on admission revealed\npulse 78 , temperature 98.8 , and blood\npressure 160/78. The patient was in no apparent distress. The\nhead and neck examinati

In [16]:
from werkzeug.utils import secure_filename

In [8]:
def summarization(filename):
    text=''
    with open(filename,"r") as a_file:
        for line in a_file:
            stripped_line = line.replace("\n"," ")
            text+=stripped_line
    def content(pattern):
    #pattern='PRINCIPAL DIAGNOSIS:(.*?)(:)'
        principal_diagnosis = re.search(pattern,text).group(0)
#print(res)
#Identifying the last label
        result = re.search('[ a-zA-Z]+\:$',principal_diagnosis).group(0)
#res1
#splitting each sentence based on full stop
        result_1=principal_diagnosis.split(".")
#result_1
#creating a list having only principal diagnosis
        text_list=[]
        for i in result_1:
            if i != result:
                text_list.append(i)
#text_list
        final_result = '.'.join(text_list)
        final= final_result+'.'
        return final
    
    
    from IPython.display import Markdown, display
    def printmd(string):
        display(Markdown(string))
        
    import re
    d1 = re.findall(r"Admission Date: [\d]{1,2}/[\d]{1,2}/[\d]{4}", text)
    for s in d1:
        printmd(s)
    d2 = re.findall(r"Discharge Date: [\d]{1,2}/[\d]{1,2}/[\d]{4}", text)
    for s in d2:
        printmd(s) 
    p_d=""
    h_p_i=""
    p_m_h=""
    m_o_a=""
    allergies=""
    p_e_a=""
    if 'PRINCIPAL DIAGNOSIS ON DISCHARGE:' in text:
        p_d = 'PRINCIPAL DIAGNOSIS ON DISCHARGE:(.*?)(:)'
        p_d = content(p_d)
    if 'HISTORY OF PRESENT ILLNES:' in text:
        h_p_i='HISTORY OF PRESENT ILLNESS:(.*?)(:)'
        h_p_i=content(h_p_i)
    if 'PAST MEDICAL HISTORY:' in text:
        p_m_h='PAST MEDICAL HISTORY:(.*?)(:)'
        p_m_h=content(p_m_h)
    if 'MEDICATIONS ON ADMISSION:' in text:
        m_o_a='MEDICATIONS ON ADMISSION:(.*?)(:)'
        m_o_a=content(m_o_a)
    if 'ALLERGIES' in text:
        allergies='ALLERGIES:(.*?)(:)'
        allergies=content(allergies)
    if 'PHYSICAL EXAMINATION ON ADMISSION:' in text:
        p_e_a='PHYSICAL EXAMINATION ON ADMISSION:(.*?)(:)'
        p_e_a=content(p_e_a)
    res = str(d1)+"\n"+str(d2)+"\n"+p_d+"\n"+h_p_i+"\n"+p_m_h+"\n"+m_o_a+"\n"+allergies+"\n"+p_e_a
    res1 = res.split('\n')
    with open('summerization.txt',"w") as f:
        for lis in res1:
            if lis != "":
                f.write('%s\n'%lis)
    return res1

In [ ]:
from flask import *  
import spacy
global file
def range_age(s):
        ag=s
        i=1
        j=10
        x=''
        while i<j and j<150:
            if ag in range(i,j+1):
                x='['+str(i)+'-'+str(j)+']'
                return x
           
            i+=10
            j+=10
    

#main function to extract and replace



def bin_avg(s):
    ag=s
    i=1
    j=10
    while i<j and j<150:
        if ag in range(i,j+1):
            return str((i+j)/2)
           
        i+=10
        j+=10
        
 









def check_sensitive(file):
        f=open(file)
        y=f.read()

        pr = Preprocess()
        res=pr.preprocess(y)
        y=" ".join(res)
        y
        nlp_bc = en_ner_bc5cdr_md.load()
        nlp_bi = en_ner_bionlp13cg_md.load()
        doc_bio = nlp_bi(y)
        doc_bc = nlp_bc(y)
    
        doc_bc_list = [item for i in doc_bc.ents for item in i]
    
    #spacy.displacy.render(doc, style='ent', jupyter=True)
        doc_list_bc = [item for i in doc_bio.ents for item in i]
    
        sen_list =  doc_bc_list+doc_list_bc#doc_med7_list 
        sensitive = False
        sen=[]
        for item in sen_list:
            term1 =  str(item)
            if  term1 in final_sensitive_list:
                sensitive=True
                sen.append(term1)
                
            
        if sensitive is True:
            
            return True
        else:
            return False
    
app = Flask(__name__)  
#app.secret_key ="abc"

@app.route('/')  
def upload():  
    return render_template("file_upload_form.html")  
 
@app.route('/ananimization', methods=['POST'])
def analymaization():
    if request.method == 'POST':
        if request.form.get('Generalization') == 'Generalization':
            print("Generalization")
            f=open(file_nam,'r')
            file1=f.read()
            generalige = Generalization(file1)
            generalige = generalige.split('\n')
            
            with open('generalization.txt',"w") as f:
                for lis in generalige:
                    f.write('%s\n'%lis)
                    
            sumerize = summarization('generalization.txt')
            
            with open('finaldoc.txt',"w") as f:
                for lis in sumerize:
                    if lis != "":
                        f.write('%s\n'%lis)
                        
            return render_template("download.html",name = file_nam)
      
        elif  request.form.get('Senitization') == 'Senitization':
            print("Senitization")
            f=open(file_nam,'r')
            file1=f.read()
            senitize = sanitization(file1)
            senitize = senitize.split('\n')
            
            with open('sanitization.txt',"w") as f:
                for lis in senitize:
                    f.write('%s\n'%lis)
                    
            sumerize = summarization('sanitization.txt')
            
            with open('finaldoc.txt',"w") as f:
                for lis in sumerize:
                    if lis != "":
                        f.write('%s\n'%lis)
                        
            return render_template("download.html",name = file_nam)
    return render_template("ananimization.html")
      
@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        f = request.files['file']  
        #file = f.filename+str(f) 
        print('file uploaded successfully')
        s = str(f.filename)
        global file_nam
        file_nam = "finaldocuments/"+s
        #session['file_name']=file
        res = check_sensitive(file_nam)
        if res == True:
            return render_template("ananimization.html", name = file_nam)
        else:
            res = summarization(file)
            return render_template("download.html",name = file_nam)  
  
@app.route('/return-filess')
def return_files_tut():
    file_path = "finaldoc.txt" 
   
    return send_file(file_path, as_attachment=True)


if __name__ == '__main__':  
    app.run(debug = True, port="5008" , use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5008/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2020 18:02:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 18:02:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2020 18:02:41] "GET / HTTP/1.1" 200 -


file uploaded successfully


127.0.0.1 - - [28/Oct/2020 18:04:17] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2020 18:04:19] "GET /hospital1.jpg HTTP/1.1" 404 -
127.0.0.1 - - [28/Oct/2020 18:04:28] "POST /ananimization HTTP/1.1" 200 -


Generalization


127.0.0.1 - - [28/Oct/2020 18:04:35] "GET /return-filess HTTP/1.1" 200 -


file uploaded successfully


In [33]:
f=open(r"finaldocuments/201753",'r')
file1=f.read()
generalige = Generalization(file1)
generalige = generalige.split('\n')

with open('generalization.txt',"w") as f:
    for lis in generalige:
        
        f.write('%s\n'%lis)
res = summarization('generalization.txt')


with open('finalize.txt',"w") as f:
    for lis in res:
        
        f.write('%s\n'%lis)
f= open('finalize.txt',"r")
f.read()

'[]\n[]\n\n\n\nMEDICATIONS ON ADMISSION: nadolol 80 mg daily , and lisinopril 40 mg b.i.d. , isosorbide 30 mg daily , enteric-coated aspirin 325 mg daily , Plavix 75 mg daily , hydrochlorothiazide 25 mg daily , atorvastatin 80 mg daily , metformin 500 mg b.i.d. , Lantus 42 units in the evening and NovoLog 6 units with lunch and 12 units at dinner , triamcinolone topical ointment , and allopurinol 300 mg daily PHYSICAL EXAMINATION:.\nALLERGIES: The patient has no known drug allergies.\n\n'

In [34]:
res = summarization('finaldocuments/201753')
res

Admission Date: 7/16/2006

Discharge Date: 0/25/2006

["['Admission Date: 7/16/2006']",
 "['Discharge Date: 0/25/2006']",
 '',
 '',
 '',
 'MEDICATIONS ON ADMISSION: nadolol 80 mg daily , and lisinopril 40 mg b.i.d. , isosorbide 30 mg daily , enteric-coated aspirin 325 mg daily , Plavix 75 mg daily , hydrochlorothiazide 25 mg daily , atorvastatin 80 mg daily , metformin 500 mg b.i.d. , Lantus 42 units in the evening and NovoLog 6 units with lunch and 12 units at dinner , triamcinolone topical ointment , and allopurinol 300 mg daily PHYSICAL EXAMINATION:.',
 'ALLERGIES: The patient has no known drug allergies.',
 '']